In [2]:
import sys
sys.path.append('..')
from utils import *
sys.path.append('../attacks/')
from ifgm import *
from ifgsm import *
from torch import nn,norm
import torch
from torchvision import transforms, datasets
import numpy as np
from tqdm import tqdm
import cv2
import pandas as pd
import numpy as np
sys.path.append('../models/')
from alexnet import *
from densenet_201 import *
from mobilenet_v2 import *
from resnet_50 import *
from vgg_19 import *

In [3]:
imgs = datasets.ImageFolder('/home/hancy/dataset/imagenet3000/',transform=transforms.ToTensor())

In [4]:
wm_origin = cv2.imread('../img/logo.jpg')
wm_origin = cv2.cvtColor(wm_origin,cv2.COLOR_BGR2RGB)
wm_origin = transforms.ToTensor()(wm_origin).cuda()

In [10]:
def attack_and_record(filename: str,model: nn.Module, imgs:datasets, wm_origin: Tensor):
    block_size = 8
    alpha = 0.1
    beta = 10/255
    eps = 20/255
    steps = 10
    l1 = 0.01
    l2 = 0.01
    s_a = 0.0005
    s_b = 0.0001
    N = 20
    beta_max = 20/255
    with open(filename,'a') as file:
        for i in tqdm(range(len(imgs)), desc='Processing'):
            img = imgs[i][0].cuda()
            img = addborder(img,block_size)
            wm = transforms.Resize(img.size()[-2:])(wm_origin)
            target = torch.tensor([imgs[i][1]]).cuda()
            pred_label = model(img.unsqueeze(0)).argmax().item()
            perd_img,wm_extracted,a_res,b_res = ifgsm_wm_opti(img,target,wm,model,alpha,beta,block_size,steps,eps,N,l1,l2,s_a,s_b,beta_max)
            res = model(perd_img.unsqueeze(0))
            perd_label = res.argmax().item()
            wm_l2_norm = norm(wm_extracted-wm).item()
            img_l2_norm = norm(perd_img-img).item()
            file.write('{},{},{},{},{},{},{},{}\n'.format(i,imgs[i][1],pred_label,perd_label,wm_l2_norm,img_l2_norm,a_res,b_res))

In [7]:
def check_result(filename,direct_mode=False):
    cols = ['index','label','pred_label','perd_label','wm_l2','img_l2','alpha','beta']
    if(direct_mode):
        cols = ['index','label','pred_label','perd_label','wm_l2','img_l2']
    res = pd.read_csv(filename,names=cols,header=None)
    print((res['label'] == res['pred_label']).sum())
    print((res['label'] == res['perd_label']).sum())
    print(res['wm_l2'].sum() / res['wm_l2'].count())
    print(res['img_l2'].sum() / res['img_l2'].count())

In [11]:
def attack_and_record_direct(filename: str,model: nn.Module, imgs:datasets, wm_origin: Tensor):
    block_size = 8
    alpha = 0.1
    beta = 10/255
    eps = 20/255
    steps = 10
    with open(filename,'a') as file:
        for i in tqdm(range(len(imgs)), desc='Processing'):
            img = imgs[i][0].cuda()
            img = addborder(img,block_size)
            wm = transforms.Resize(img.size()[-2:])(wm_origin)
            target = torch.tensor([imgs[i][1]]).cuda()
            pred_label = model(img.unsqueeze(0)).argmax().item()
            perd_img = ifgm_direct(img,target,wm,model,alpha,beta,block_size,steps,eps)
            wm_extracted = extract_wm(img,perd_img,alpha,block_size)
            res = model(perd_img.unsqueeze(0))
            perd_label = res.argmax().item()
            wm_l2_norm = norm(wm_extracted-wm).item()
            img_l2_norm = norm(perd_img-img).item()
            file.write('{},{},{},{},{},{}\n'.format(i,imgs[i][1],pred_label,perd_label,wm_l2_norm,img_l2_norm))

In [6]:
model = resnet50_IN().cuda().eval()

In [11]:
def embed_and_record(filename: str,model: nn.Module, imgs:datasets, wm_origin: Tensor):
    block_size = 8
    alpha = 0.1
    with open(filename,'a') as file:
        for i in tqdm(range(len(imgs)), desc='Processing'):
            img = imgs[i][0].cuda()
            img = addborder(img,block_size)
            wm = transforms.Resize(img.size()[-2:])(wm_origin)
            wmed_img = embed_wm(img,wm,alpha,block_size)
            target = torch.tensor([imgs[i][1]]).cuda()
            pred_label = model(wmed_img.unsqueeze(0)).argmax().item()
            wm_extracted = extract_wm(img,wmed_img,alpha,block_size)
            wm_l2_norm = norm(wm_extracted-wm).item()
            img_l2_norm = norm(wmed_img-img).item()
            file.write('{},{},{},{},{}\n'.format(i,imgs[i][1],pred_label,wm_l2_norm,img_l2_norm))

In [13]:
filename = '/home/hancy/code/adv_wm/atk_result/10131200_vgg19_wmed_imnet.txt'
embed_and_record(filename,model,imgs,wm_origin)

Processing: 100%|██████████| 3000/3000 [05:48<00:00,  8.62it/s]


In [35]:
filename = '/home/hancy/code/adv_wm/atk_result/10131200_resnet50_ifgm_direct_imnet.txt'
attack_and_record_direct(filename,model,imgs,wm_origin)
check_result(filename)

Processing: 100%|██████████| 3000/3000 [12:25<00:00,  4.02it/s]

2489
1
51.82712747595235
16.17636101682981


In [12]:
filename = '/home/hancy/code/adv_wm/atk_result/10141000_resnet50_ifgm_direct_imnet.txt'
attack_and_record_direct(filename,model,imgs,wm_origin)
check_result(filename)

Processing: 100%|██████████| 3000/3000 [12:28<00:00,  4.01it/s]

2489
1
51.86286733804029
16.177583479007087


In [13]:
filename = '/home/hancy/code/adv_wm/atk_result/10141000_resnet50_ifgsm_wm_imnet.txt'
attack_and_record(filename,model,imgs,wm_origin)
check_result(filename)

Processing: 100%|██████████| 3000/3000 [1:03:39<00:00,  1.27s/it]

2489
101
265.705653318724
24.74714222129186


In [15]:
filename = '/home/hancy/code/adv_wm/atk_result/10131200_vgg19_wmed_imnet.txt'
cols = ['index','label','pred_label','wm_l2','img_l2']
res = pd.read_csv(filename,names=cols,header=None)
print((res['label'] == res['pred_label']).sum())
print(res['wm_l2'].sum() / res['wm_l2'].count())
print(res['img_l2'].sum() / res['img_l2'].count())

2267
46.10391305866676
16.044131396929423
